In [1]:
%pip install -r ~/code/GitHub/UPennContrast/devops/girder/annotation_client/requirements.txt
%pip install -e ~/code/GitHub/UPennContrast/devops/girder/annotation_client/
# %pip install pandas
# %pip install imageio
# %pip install rtree
# %pip install scikit-image
# %pip install shapely
# %pip install nd2reader

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Applications/Xcode.app/Contents/Developer/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Obtaining file:///Users/arjunraj/code/GitHub/UPennContrast/devops/girder/annotation_client
  Attempting uninstall: annotation-client
    Found existing installation: annotation-client 0.1
    Uninstalling annotation-client-0.1:
      Successfully uninstalled annotation-client-0.1
  Running setup.py develop for annotation-client
You should consider upgrading via the '/Applications/Xcode.app/Contents/Developer/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import girder_client
import annotation_client.annotations as annotations
import annotation_client.tiles as tiles
import pandas as pd
import annotation_tools
#import imageio
from shapely.geometry import Point, Polygon, LineString
#pd.set_option('display.max_colwidth', None)
import numpy as np

/Users/arjunraj/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2.0 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [3]:
apiUrl = 'http://localhost:8080/api/v1'
username = 'admin'
password = 'password'

In [4]:
client = girder_client.GirderClient(apiUrl=apiUrl)
user_id = client.authenticate(username=username, password=password)['_id']
annotationClient = annotations.UPennContrastAnnotationClient(apiUrl=apiUrl, token=client.token)

In [5]:
datasets = dict()
for folder in annotationClient.client.listFolder(user_id,'user'):
    for dataset in annotationClient.client.listFolder(folder['_id']):
        datasets[dataset['name']] = dataset['_id']

pd.DataFrame(datasets.keys(),columns=['Dataset Name'])

,Dataset Name
0,111111_20210723_155544_798__Plate20210723_STR_...
1,111_20200110_2Dent_001
2,20200110_2Dent_001_maxmerge
3,"20221122_143426_960__WellA4_ChannelDAPI,CY3,A5..."
4,"20230314_175616_594__WellB2_ChannelDAPI,YFP,CY..."
...,...
135,UNROLL3_MS2_1_maxmerge
136,UNROLL_MS2_1
137,UNROLL_MS2_1_maxmerge
138,"WellA1_ChannelBrightfield,YFP,CY5,DAPI,A594_Se..."


In [6]:
dataset_name = 'Prop_test_normmedia_8well_col2_livecellgfp'

In [7]:
datasetId = datasets[dataset_name]
datasetClient = tiles.UPennContrastDataset(apiUrl=apiUrl, token=client.token, datasetId=datasetId)
print(datasetClient.tiles)
tile_width = datasetClient.tiles['tileWidth']
tile_height = datasetClient.tiles['tileHeight']
print(tile_width)
print(tile_height)

# test of getting tile metadata
internalMetadata = datasetClient.tilesInternal
import pprint
pprint.pprint(internalMetadata)

{'IndexRange': {'IndexC': 2, 'IndexT': 4, 'IndexXY': 6, 'IndexZ': 7}, 'IndexStride': {'IndexC': 1, 'IndexT': 14, 'IndexXY': 56, 'IndexZ': 2}, 'bandCount': 1, 'channelmap': {'Brightfield': 0, 'YFP': 1}, 'channels': ['Brightfield', 'YFP'], 'dtype': 'uint16', 'frames': [{'Channel': 'Brightfield', 'Frame': 0, 'Index': 0, 'IndexC': 0, 'IndexT': 0, 'IndexXY': 0, 'IndexZ': 0}, {'Channel': 'YFP', 'Frame': 1, 'Index': 0, 'IndexC': 1, 'IndexT': 0, 'IndexXY': 0, 'IndexZ': 0}, {'Channel': 'Brightfield', 'Frame': 2, 'Index': 1, 'IndexC': 0, 'IndexT': 0, 'IndexXY': 0, 'IndexZ': 1}, {'Channel': 'YFP', 'Frame': 3, 'Index': 1, 'IndexC': 1, 'IndexT': 0, 'IndexXY': 0, 'IndexZ': 1}, {'Channel': 'Brightfield', 'Frame': 4, 'Index': 2, 'IndexC': 0, 'IndexT': 0, 'IndexXY': 0, 'IndexZ': 2}, {'Channel': 'YFP', 'Frame': 5, 'Index': 2, 'IndexC': 1, 'IndexT': 0, 'IndexXY': 0, 'IndexZ': 2}, {'Channel': 'Brightfield', 'Frame': 6, 'Index': 3, 'IndexC': 0, 'IndexT': 0, 'IndexXY': 0, 'IndexZ': 3}, {'Channel': 'YFP', 'F

In [49]:
pointAnnotationList = annotationClient.getAnnotationsByDatasetId(datasetId, shape='point')
blobAnnotationList = annotationClient.getAnnotationsByDatasetId(datasetId, shape='polygon')
lineAnnotationList = annotationClient.getAnnotationsByDatasetId(datasetId, shape='line')

In [41]:
print(len(blobAnnotationList))
print(len(pointAnnotationList))
print(len(lineAnnotationList))
print(blobAnnotationList[0])
print(pointAnnotationList[0])
print(lineAnnotationList[0])

7
21
8
{'_id': '64aa9942f69bc320e7503826', 'access': {'groups': [], 'users': [{'flags': [], 'id': '62891fc02060d362984bc104', 'level': 2}]}, 'channel': 1, 'coordinates': [{'x': 313.2846560841441, 'y': 436.8299470906123}, {'x': 313.2846560841441, 'y': 439.44455026536895}, {'x': 312.70363315642044, 'y': 442.0591534401256}, {'x': 311.25107583711116, 'y': 443.8022222232967}, {'x': 309.50800705394005, 'y': 445.254779542606}, {'x': 307.47442680690705, 'y': 447.57887125350084}, {'x': 306.02186948759777, 'y': 449.32194003667195}, {'x': 303.4072663128411, 'y': 450.1934744282575}, {'x': 298.7590828910515, 'y': 450.1934744282575}, {'x': 296.1444797162948, 'y': 449.32194003667195}, {'x': 293.5298765415381, 'y': 447.86938271736267}, {'x': 292.0773192222289, 'y': 446.7073368619153}, {'x': 291.2057848306433, 'y': 444.96426807874417}, {'x': 290.91527336678143, 'y': 442.0591534401256}, {'x': 291.2057848306433, 'y': 440.60659612081633}, {'x': 292.3678306860907, 'y': 437.70148148219783}, {'x': 292.948853

In [17]:
filteredPointList = annotation_tools.get_annotations_with_tags(pointAnnotationList,['outerPt'],exclusive=False)
print(len(filteredPointList))
targetPointList = annotation_tools.get_annotations_with_tags(pointAnnotationList,['innerPt'],exclusive=False)
print(len(targetPointList))

8
10


In [29]:
import math

def calculate_distance(point1, point2):
    x1, y1, z1 = point1['coordinates'][0]['x'], point1['coordinates'][0]['y'], point1['coordinates'][0]['z']
    x2, y2, z2 = point2['coordinates'][0]['x'], point2['coordinates'][0]['y'], point2['coordinates'][0]['z']
    return math.sqrt((x2 - x1) ** 2 + (y2 - y1) ** 2 + (z2 - z1) ** 2)


def find_matching_annotations_by_location(source, target_list, Time=True, XY=True, Z=True):
    """
    This function filters the target_list based on the 'location' of the source point.
    The function parameters 'Time', 'XY', and 'Z' can be set to True or False to specify whether these 'location' attributes need to be matched.
    By default, all of these parameters are set to True, meaning all 'location' attributes need to match.

    Parameters:
    source (dict): The source point annotation object
    target_list (list): The list of target point annotation objects
    Time (bool): Specifies whether the 'Time' attribute of 'location' needs to be matched. Default is True.
    XY (bool): Specifies whether the 'XY' attribute of 'location' needs to be matched. Default is True.
    Z (bool): Specifies whether the 'Z' attribute of 'location' needs to be matched. Default is True.

    Returns:
    list: The filtered list of target point annotation objects that match the specified 'location' attributes

    Example of usage:
    1) Matching all 'location' attributes:
    source = {...}  # source point annotation object
    target_list = [...]  # target point annotation list
    matching_annotations = find_matching_annotations_by_location(source, target_list)

    2) Matching specified 'location' attributes (in this case, 'Time' and 'XY'):
    source = {...}  # source point annotation object
    target_list = [...]  # target point annotation list
    matching_annotations = find_matching_annotations_by_location(source, target_list, Time=True, XY=True, Z=False)
    """
    params = {'Time': Time, 'XY': XY, 'Z': Z}
    return [target for target in target_list if all(source['location'].get(attr) == target['location'].get(attr) for attr, value in params.items() if value)]

def find_closest_point(source, target_list):
    min_distance = float('inf')
    closest_point = None
    for target in target_list:
        if source['_id'] == target['_id']:
            continue
        distance = calculate_distance(source, target)
        if distance < min_distance:
            min_distance = distance
            closest_point = target
    return closest_point, min_distance

#filteredPointList = [...]  # Your filteredPointList here
#targetPointList = [...]  # Your targetPointList here

for source in filteredPointList:
    # Add the attributes you want to match as keyword arguments here. If no arguments are provided, all 'location' attributes will be matched.
    matching_annotations = find_matching_annotations_by_location(source, targetPointList, Time=True, XY=True)
    closest_point, min_distance = find_closest_point(source, matching_annotations)
    print(f"For point id {source['_id']}, closest point is {closest_point['_id']} with distance {min_distance}")

for source in filteredPointList:
    # Add the attributes you want to match as keyword arguments here. If no arguments are provided, all 'location' attributes will be matched.
    matching_annotations = find_matching_annotations_by_location(source, targetPointList)
    closest_point, min_distance = find_closest_point(source, matching_annotations)
    print(f"For point id {source['_id']}, closest point is {closest_point['_id']} with distance {min_distance}")



For point id 64aa9970f69bc320e7503830, closest point is 64aa99e4f69bc320e7503858 with distance 44.97019871692391
For point id 64aa9971f69bc320e7503832, closest point is 64aa99e4f69bc320e7503858 with distance 40.542745732502006
For point id 64aa9972f69bc320e7503834, closest point is 64aa99ecf69bc320e7503864 with distance 22.147569722466812
For point id 64aa9972f69bc320e7503836, closest point is 64aa99e3f69bc320e7503856 with distance 23.26450919111039
For point id 64aa9973f69bc320e7503838, closest point is 64aa99e4f69bc320e7503858 with distance 15.882769465319464
For point id 64aa9974f69bc320e750383a, closest point is 64aa99e4f69bc320e7503858 with distance 36.323228159944755
For point id 64aa9990f69bc320e7503852, closest point is 64aa9a46f69bc320e750386c with distance 7.65561016396188
For point id 64aa9aa7f69bc320e7503876, closest point is 64aa9aa1f69bc320e7503870 with distance 10.806361218119244
For point id 64aa9970f69bc320e7503830, closest point is 64aa99e4f69bc320e7503858 with distan

TypeError: 'NoneType' object is not subscriptable

In [28]:
print(closest_point)
print(min_distance)

None
inf


In [25]:
import math

def calculate_distance(p1, p2):
    return math.sqrt((p2['x'] - p1['x'])**2 + (p2['y'] - p1['y'])**2 + (p2['z'] - p1['z'])**2)

def total_length(line):
    coordinates = line['coordinates']
    return sum(calculate_distance(coordinates[i], coordinates[i+1]) for i in range(len(coordinates)-1))

myLine = lineAnnotationList[2]
print(total_length(myLine))


164.89992701209385


Now let's try to push a new annotation

In [40]:
# Make a small square at the top left of the image
square_coords = [(10, 10),(30, 10),(30, 30),(10, 30)]

newAnnotation = {
    "tags": ['newTag'], # *** NEED TO UPDATE TO ADD A NEW TAG ****
    "shape": "polygon",
    "channel": blobAnnotationList[0]['channel'],
    "location": blobAnnotationList[0]['location'],
    "datasetId": blobAnnotationList[0]['datasetId'],
    "coordinates": [{"x": float(coord[0]), "y": float(coord[1])} for coord in square_coords]
}
annotationClient.createAnnotation(newAnnotation)


{'_id': '64a7006f1ed2b05b5e1d8b83',
 'access': {'groups': [],
  'users': [{'flags': [], 'id': '62891fc02060d362984bc104', 'level': 2}]},
 'channel': 1,
 'coordinates': [{'x': 10.0, 'y': 10.0},
  {'x': 30.0, 'y': 10.0},
  {'x': 30.0, 'y': 30.0},
  {'x': 10.0, 'y': 30.0}],
 'datasetId': '649b6efb5d422edecab7f9d8',
 'location': {'Time': 0, 'XY': 0, 'Z': 0},
 'shape': 'polygon',
 'tags': ['newTag']}